<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/LooseThreshold__tfidf_consine_dedupedExactMatch_OnlyEnglish_50k_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'

Mounted at /content/drive


In [2]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-2-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [ ]:
df_english_titles=df_english_titles[~df_english_titles['title'].str.lower().isin(["conclusion", "poetry","introduction","front matter","editorial","back matter","correspondence","ist of Abbreviations","Book Reviews","Notifiable Diseases and Mortality Tables"])]
len(df_english_titles)

47526

In [ ]:
df_english_titles.to_csv(os.path.join(directory, '50k_sample.csv'), index=False)

In [ ]:
len(df_english_titles)/50000

0.95052

# from here

In [2]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [3]:
df_english_titles['title'] = df_english_titles['title'].str.lower()
duplicate_rows = df_english_titles[df_english_titles.duplicated(subset='title')]

df_english_titles_temp = df_english_titles.drop(duplicate_rows.index)
df_english_titles=df_english_titles_temp

In [4]:

import nltk
nltk.download('punkt')
from nltk import word_tokenize

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
df_english_titles['title'] = df_english_titles['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df_english_titles['title'] = df_english_titles['title'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
len(df_english_titles)

47377

# TF-IDF and consine similarity

In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_english_titles['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

num_titles = len(df_english_titles)
for i in range(num_titles):
    current_title = df_english_titles.iloc[i]['title']
    similarities = cosine_sim[i]

    for j in range(num_titles):
        if i != j:
            other_title = df_english_titles.iloc[j]['title']
            similarity_score = similarities[j]


            if similarity_score > 0.75  :
                print(f"{current_title} & {other_title} (score: {similarity_score:.4f})")


inequality & income inequality (score: 0.7524)
how does country risk matter for foreign direct investment? & country risk and foreign direct investment (score: 0.7534)
editor's introduction & guest editor's introduction (score: 0.7580)
could the corporations become engines for international cooperation? & could the corporations become engines for international cooperation (score: 1.0000)
pengaruh profitabilitas, struktur aktiva, dan ukuran perusahaan terhadap struktur modal & pengaruh profitabilitas, pertumbuhan, struktur aktiva dan ukuran perusahaan terhadap struktur modal ( pada perusahaan property dan real estate yang terdaftar di bei) (score: 0.8089)
labour costs and employment policy & labour and employment (score: 0.7501)
the institutional economics of foreign aid & the institutional economics of foreign aid: introduction (score: 0.8861)
a valuation model for credit default swaps & credit default swaps (score: 0.8274)
an introduction to the constitution of india & introduction to

14h 32min 16s

# use this to drop punctuation marks and run with condition on word length

In [5]:
!pip install stopwords

In [6]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
def remove_punctuation_nltk(text):
  """Removes punctuation marks and stop words from a string (optional)."""
  stop_words = stopwords.words('english')
  words = [word for word in text.lower().split() if word not in stop_words and word.isalpha()]
  return ' '.join(words)

df_english_titles['title_no_punct'] = df_english_titles['title'].apply(remove_punctuation_nltk)




In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Filter titles based on length before vectorization
filtered_titles = df_english_titles[df_english_titles['title_no_punct'].apply(lambda t: len(t.split()) >= 2 and len(t) >= 10)]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_titles['title_no_punct'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Use the length of the filtered DataFrame for comparison
num_titles = len(filtered_titles)

for i in range(num_titles):
    current_title = filtered_titles.iloc[i]['title_no_punct']
    similarities = cosine_sim[i]

    for j in range(num_titles):
        if i != j:
            other_title = filtered_titles.iloc[j]['title_no_punct']
            similarity_score = similarities[j]

            if similarity_score > 0.8:
                print(f"{current_title} & {other_title} (score: {similarity_score:.4f})")
                print('_' * 100)

book review welfare & book review (score: 0.8223)
____________________________________________________________________________________________________
book review welfare & book review (score: 0.8223)
____________________________________________________________________________________________________
economic challenge & challenge economic social (score: 0.8410)
____________________________________________________________________________________________________
could corporation become engine international & could corporation become engine international cooperation (score: 0.9327)
____________________________________________________________________________________________________
economic success & economic success (score: 1.0000)
____________________________________________________________________________________________________
pengaruh struktur dan ukuran perusahaan terhadap struktur modal & pengaruh struktur modal dan struktur kepemilikan terhadap nilai perusahaan (score: 0.8183)
_

protection united kingdom & united kingdom (score: 0.8159)
		____________________________________________________________________________________________________
		CPU times: user 20h 59min 23s, sys: 1min 39s, total: 21h 1min 3s
		Wall time: 21h 21s